# Program : Amazon Proudct Review Extractor

- Objective of this program is to extract all the reivew details for the amazon product
- main function to be called with url as input
- Input url should be product page url
- Before running the program, chrome driver path (funct: main , var : ch_driver_path) need to be changed as its specific to system.


## Amazon product page pattern observed to implement in program 
- Actual review page get directed by clicking 'See all reviews'
- One review page contains maximum 10 reviews. 
- Global review count on review page used to determine total review pages
- Next review page url get appended with page number


### Function description 
- get_page_count : Derive total number of page based on Global review input.
- get_url : This is to derive next review page, it takes page number and global review text as input 
- get_review_extracts : Function derives  all element of review.   

## Outcome 
- This will extract all reviews details against the product and store in csv path. 

In [10]:
from bs4 import BeautifulSoup
from selenium import webdriver
import math
import pandas as pd 
import time

# Derive the page number based on total number of review 

def get_page_count(soup_obj):
    total_reviews_str = soup_obj.find('div', {'data-hook': 'cr-filter-info-review-rating-count'})
    total_reviews_str = total_reviews_str.text.strip()
    total_reviews_str = total_reviews_str.split('|')
    total_reviews_str =  [k for k in total_reviews_str if 'global reviews' in k]
    total_reviews_str
    total_reviews = int(''.join(filter(str.isdigit, total_reviews_str[0])))
    #Get page count 
    page_count = int(math.ceil(total_reviews/10))
    
    return page_count 

## Function to derive URL for the other review pages 
def get_url(template , pageNumber):
    """Generate url against page number"""
    template += '&pageNumber={}'
    url = template.format(pageNumber)
    return url

# Extract each review componet 
def get_review_extracts(soup_obj):
    review_page_results = soup_obj.find_all('div' , {'data-hook' : 'review'})
    #print(review_page_results)
    review_details_col = ["Title", "Rating", "Date", "Text", "Found_Helpful"]
    # Initalize empty DataFrames
    review_df = pd.DataFrame(None, columns = review_details_col)
    for item in review_page_results : 
        
    #     print(item)
        #record = extract_record(item)
        
        ## --------------- Start of Extraction --------------------##
      
        try:
            review_title = item.find('span' , 'a-profile-name').text
        except AttributeError:
            review_title = ''   
        try:      
            review_rating = item.find(attrs={"data-hook": "review-star-rating"}).text
        except AttributeError:
            review_rating = '' 
        try:     
            review_Date =item.find(attrs={"data-hook": "review-date"}).text   
        except :
            review_Date = ''
        try:     
            review_text = item.find("span",{"data-hook": "review-body"}).text.strip() 
        except AttributeError:
            return
        try: 
            review_help = item.find(attrs={"data-hook": "helpful-vote-statement"}).text
        except AttributeError:
            review_help = ''   

        # Post details
        review_details = {
                        'Title': review_title,
                        'Rating': review_rating,
                        'Date': review_Date,
                        'Text': review_text,
                        'Found_Helpful':review_help
                        }    
        if review_details :
            review_df = review_df.append(review_details, ignore_index=True)
     
    return  review_df   

def main(p_url):


    ## Define and Initialize the result daaframe
    result_details_col = ["Title", "Rating", "Date", "Text", "Found_Helpful"]
    result_df = pd.DataFrame(None, columns = result_details_col)


    #Load the driver with the given  URL 
    ## Please note the driveer path should be specific to user 
    ch_driver_path = r'C:/Users/Amrendra Singh/JypyterProject/WebScrapping/chromedriver.exe'
    driver = webdriver.Chrome(executable_path = ch_driver_path )  

    #url = 'https://www.amazon.com/Samsung-Unlocked-Fingerprint-Recognition-Long-Lasting/dp/B082XXKZRC/ref=cm_cr_arp_d_product_top?ie=UTF8'
    driver.get(p_url)

    # From the page Inspect get the Xpath of See All comment 
    review_xpath = '//*[@id="reviews-medley-footer"]/div[2]/a'


    review_page_first = driver.find_element_by_xpath(review_xpath).get_property('href')
    ## print(review_page_first)
    driver.get(review_page_first)

    ## Time to sleep 
    time.sleep(1) 

    soup = BeautifulSoup(driver.page_source , 'html.parser')
    ## Determinte the page count 
    page_count = get_page_count(soup)
    # print('Page count ', page_count)
    ### Origin of the the review
    ##review_origin = soup.find('h3' , {'data-hook' : 'arp-local-reviews-header'}).text.strip()

    # Now work for the getting the review details on the page 
    record = []

    #review_details = []
    result_df = result_df.append(get_review_extracts(soup))
    # review_details_df.append(result_df)

    #Other pages review 
    for page in range(2, page_count+1 ):  
            print('Page Number ', page)
            review_pages_url = get_url(review_page_first , page)
            print('Page_url', review_pages_url)
            driver.get(review_pages_url)

            ## Time to sleep 
            time.sleep(1) 
            soup = BeautifulSoup(driver.page_source , 'html.parser')
            result_df = result_df.append(get_review_extracts(soup))
    #         review_details_df.append(result_df)

    #Close the driver. 
    driver.close() 
    result_df.to_csv ('Review_All_Amazon_Product.csv', index = False, header=True)


## Main fuction to be called 
- url should be of any amazon product page. 
- Result will be saved in its root folder with name Review_All_Amazon_Product
- For another run, make sure csv file not exists in root folder

In [9]:
input_url = https://www.amazon.com/Samsung-Unlocked-Fingerprint-Recognition-Long-Lasting/dp/B082XXKZRC/ref=cm_cr_arp_d_product_top?ie=UTF8
main(input_url)

Page Number  2
Page_url https://www.amazon.com/Apple-MacBook-13-inch-512GB-Storage/product-reviews/B08637XBML/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=2
Page Number  3
Page_url https://www.amazon.com/Apple-MacBook-13-inch-512GB-Storage/product-reviews/B08637XBML/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=3
Page Number  4
Page_url https://www.amazon.com/Apple-MacBook-13-inch-512GB-Storage/product-reviews/B08637XBML/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=4
Page Number  5
Page_url https://www.amazon.com/Apple-MacBook-13-inch-512GB-Storage/product-reviews/B08637XBML/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=5
Page Number  6
Page_url https://www.amazon.com/Apple-MacBook-13-inch-512GB-Storage/product-reviews/B08637XBML/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=6
Page Number  7
Page_url https://www.amazon.com/Apple-MacBook-13-inch-512GB-Stora

Page Number  47
Page_url https://www.amazon.com/Apple-MacBook-13-inch-512GB-Storage/product-reviews/B08637XBML/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=47
Page Number  48
Page_url https://www.amazon.com/Apple-MacBook-13-inch-512GB-Storage/product-reviews/B08637XBML/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=48
Page Number  49
Page_url https://www.amazon.com/Apple-MacBook-13-inch-512GB-Storage/product-reviews/B08637XBML/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=49
Page Number  50
Page_url https://www.amazon.com/Apple-MacBook-13-inch-512GB-Storage/product-reviews/B08637XBML/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=50
Page Number  51
Page_url https://www.amazon.com/Apple-MacBook-13-inch-512GB-Storage/product-reviews/B08637XBML/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=51
Page Number  52
Page_url https://www.amazon.com/Apple-MacBook-13-inch-